In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("rational_pred.csv")

In [5]:
del data["Unnamed: 0"]

In [50]:
data.loc[:, "weird"] = (data["is_applied"] - data["predict"] == 1) *1

In [52]:
idx = data.loc[data.weird == 1].index

In [11]:
d = pd.read_csv("learn.csv")

In [13]:
del d["Unnamed: 0"]

In [15]:
d.loc[:,"id"] = d.bank_id * 1000 + d.product_id

In [17]:
del d["loanapply_insert_time"]
del d["bank_id"]
del d["product_id"]
del d["birth_year"]
del d["gender"]
del d["houseown_type"]
del d["personal_rehabilitation_yn"]
del d["personal_rehabilitation_complete_yn"]
del d["existing_loan_cnt"]
del d["existing_loan_amt"]

In [19]:
del d["yearly_income"]

In [21]:
del d["user_id"]

In [23]:
del d["employment_type"]

In [24]:
c1 = pd.get_dummies(d.income_type)

In [25]:
d = pd.concat([d, c1], axis = 1)

In [27]:
del d["income_type"]

In [29]:
del d["EARNEDINCOME2"]
del d["FREELANCER"]
del d["OTHERINCOME"]
del d["PRACTITIONER"]
del d["PRIVATEBUSINESS"]

In [32]:
d.insert_time = pd.to_datetime(d.insert_time)
d.company_enter_month = pd.to_datetime(d.company_enter_month)

In [33]:
d

,application_id,loan_limit,loan_rate,is_applied,insert_time,credit_score,company_enter_month,desired_amount,purpose,id,EARNEDINCOME
0,566528,20000000.0,19.1,0.0,2022-03-01 00:11:35,580.0,2022-06-01,1000000.0,생활비,13123,0
1,180433,16000000.0,15.0,0.0,2022-03-01 00:12:05,740.0,2022-02-01,3000000.0,생활비,19231,0
2,180433,22000000.0,16.6,0.0,2022-03-01 00:12:05,740.0,2022-02-01,3000000.0,생활비,13262,0
3,1657888,3000000.0,13.5,0.0,2022-03-01 00:12:06,580.0,2021-02-01,3000000.0,생활비,42216,0
4,180433,3000000.0,14.9,0.0,2022-03-01 00:12:05,740.0,2022-02-01,3000000.0,생활비,1061,0
...,...,...,...,...,...,...,...,...,...,...,...
10264381,455157,3000000.0,18.0,0.0,2022-05-31 23:54:27,610.0,1997-04-01,50000000.0,대환대출,42258,1
10264382,455157,3000000.0,17.5,0.0,2022-05-31 23:54:27,610.0,1997-04-01,50000000.0,대환대출,46227,1
10264383,455157,5000000.0,18.4,0.0,2022-05-31 23:54:27,610.0,1997-04-01,50000000.0,대환대출,22221,1
10264384,455157,3000000.0,14.5,0.0,2022-05-31 23:54:27,610.0,1997-04-01,50000000.0,대환대출,38016,1


In [53]:
weird_id = d.loc[idx, "id"]

In [54]:
weird_id

6          24263
42         36157
43         55025
72         26080
80         14128
           ...  
4763008    21196
4763045    13262
4763062    35168
4763067    17019
4763077    13262
Name: id, Length: 230642, dtype: int64

In [57]:
y = d.groupby("application_id").max()

In [63]:
appyes = y.loc[y.is_applied == 1].index

In [74]:
x = d.loc[d.application_id.isin(appyes)]

In [75]:
x = x.sort_values(by = ["application_id", "loan_rate"])

In [76]:
x.loc[:, ["sum", "more", "predict"]] = 0

In [79]:
del x["sum"]
del x["more"]

In [86]:
pred = np.array(data.predict)

In [87]:
x.loc[:, "predict"] = pred

In [90]:
weird_id

6          24263
42         36157
43         55025
72         26080
80         14128
           ...  
4763008    21196
4763045    13262
4763062    35168
4763067    17019
4763077    13262
Name: id, Length: 230642, dtype: int64

In [94]:
x.loc[:, "weird"] = x.id.isin(weird_id)

In [95]:
from catboost import CatBoostClassifier, Pool

In [96]:
model = CatBoostClassifier(iterations=2,
                           depth=5,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)

In [98]:
del x["application_id"]
del x["id"]

In [113]:
del x["purpose"]

In [114]:
xxtr = x[x['insert_time']<'2022-05-16']
xxtt = x[x['insert_time']>= '2022-05-16']

In [115]:
del xxtr["insert_time"]
del xxtt["insert_time"]

In [116]:
xxtrl = xxtr["is_applied"]
xxttl = xxtt["is_applied"]

In [117]:
del xxtr["is_applied"]
del xxtt["is_applied"]

In [104]:
kk = x.is_applied.mean()

In [118]:
train = Pool(
    data= xxtr,
    label = xxtrl,
    weight = xxtrl * (1-2*kk) + kk
)

In [119]:
model.fit(train)

0:	learn: 0.5027430	total: 265ms	remaining: 265ms
1:	learn: 0.4811326	total: 371ms	remaining: 0us


In [120]:
p = model.predict(xxtt)

In [121]:
from sklearn.metrics import f1_score, confusion_matrix

In [122]:
f1_score(xxttl, p)

0.4788557213930348

In [124]:
model.get_feature_importance()

array([ 7.69868754,  4.10410306, 10.27134005,  2.18780694,  7.64741629,
        3.31213986, 64.77850625,  0.        ])

In [125]:
xxtr

,loan_limit,loan_rate,credit_score,company_enter_month,desired_amount,EARNEDINCOME,predict,weird
4955883,12000000.0,6.6,590.0,2021-06-01,20000000.0,1,1,True
4955966,12000000.0,6.6,590.0,2021-06-01,20000000.0,1,1,True
4955890,12000000.0,7.1,590.0,2021-06-01,20000000.0,1,0,True
4955887,12000000.0,7.9,590.0,2021-06-01,20000000.0,1,0,True
4955970,12000000.0,8.0,590.0,2021-06-01,20000000.0,1,0,True
...,...,...,...,...,...,...,...,...
2591389,22000000.0,16.9,560.0,2017-05-01,5000000.0,1,1,True
2591428,22000000.0,16.9,560.0,2017-05-01,5000000.0,1,0,True
2591393,13000000.0,17.4,560.0,2017-05-01,5000000.0,1,0,True
2591390,5000000.0,18.7,560.0,2017-05-01,5000000.0,1,0,True
